In [1]:
from __future__ import print_function
import time
import numpy as np

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PowerTransformer, Normalizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, Flatten, Dropout, Input
import tensorflow as tf
from sklearn.metrics import roc_auc_score

from scipy import interp

from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize, LabelBinarizer

import nltk
from nltk.cluster import KMeansClusterer
from sklearn import cluster, metrics
nltk.download('punkt')
nltk.download('stopwords')

from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support

from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import math
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

from matplotlib import rc
from matplotlib import font_manager as fm, rcParams 
import matplotlib.pyplot as plt 
plt.rc('xtick',labelsize=15)
plt.rc('ytick',labelsize=15)
plt.rcParams['figure.dpi'] = 120
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

#LatX fonts
rc('text', usetex=True)
plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}']
!apt install texlive-fonts-recommended texlive-fonts-extra cm-super dvipng

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  cm-super-minimal fonts-adf-accanthis fonts-adf-berenis fonts-adf-gillius
  fonts-adf-universalis fonts-cabin fonts-comfortaa fonts-croscore
  fonts-crosextra-caladea fonts-crosextra-carlito fonts-dejavu-core
  fonts-dejavu-extra fonts-droid-fallback fonts-ebgaramond
  fonts-ebgaramond-extra fonts-font-awesome fonts-freefont-otf
  fonts-freefont-ttf fonts-gfs-artemisia fonts-gfs-complutum fonts-gfs-didot
  fonts-gfs-neohellenic fonts-gfs-olga fonts-gfs-solomos fonts-go
  fonts-junicode fonts-lato fonts-linuxlibertine fonts-lmodern fonts-lobster
  fonts-lobstertwo fonts-noto-hinted fonts-noto-mono fonts-oflb-asana-math
  fonts

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
!pip install -U -q PyDrive
from oauth2client.client import GoogleCredentials
import warnings
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
!pip install dataframe_image
import dataframe_image as dfi

import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize, LabelBinarizer
warnings.filterwarnings('ignore')

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def build_df(file_id, file_name, index_col=None):
  downloaded = drive.CreateFile({'id':file_id})
  downloaded.GetContentFile(file_name)
  if index_col is not None :
    return pd.read_csv(file_name, index_col=index_col)
  return pd.read_csv(file_name)

df = build_df('1LEbg13-KBZzHmIWAv9SnthGUzjvcbJuA','stylo-data.csv')
news_df = build_df('15zZXsks6cI0FY4vpL555lBQSBr1QyjT4', 'news-data.csv' )
varela_df = build_df('1JoC7GlAmdQO9WPZeJZKMbv4djyQzeezC',  'varela-stylo-data.csv')
varela_df.pop('Subject')

     |████████████████████████████████| 1.3MB 10.7MB/s 
     |████████████████████████████████| 296kB 39.8MB/s 
     |████████████████████████████████| 143kB 44.4MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3163 sha256=3990c42dcbb2d076e4e1bae08c35c6deefb03a27267d4ff8e04074a9140483d0
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl


0       ASSUNTOS VARIADOS
1       ASSUNTOS VARIADOS
2       ASSUNTOS VARIADOS
3       ASSUNTOS VARIADOS
4       ASSUNTOS VARIADOS
              ...        
2995              TURISMO
2996              TURISMO
2997              TURISMO
2998              TURISMO
2999              TURISMO
Name: Subject, Length: 3000, dtype: object

In [3]:
#Some clean-up on line breaks
def clean_df(df):
  df = df.replace(to_replace ='\r', value = ' ', regex = True) 
  df = df.replace(to_replace ='\n ', value = '\n', regex = True) 
  df = df.replace(to_replace ='\n', value = ' ', regex = True)
  df = df.replace(to_replace ='\n{2,*}', value = ' ', regex = True)
  return df

#Read textual datasets
raw_df_test = build_df('1B-Ozck3R_cMeREw3yQQcODcz8UZ8Hvp7', 'raw_data.csv')
raw_df = clean_df(build_df('1B-Ozck3R_cMeREw3yQQcODcz8UZ8Hvp7', 'raw_data.csv'))
raw_news_df = clean_df(build_df('1P0UF5IRI3VNUuAulldEn84ntVfFLGiIq', 'raw_news_data.csv'))
raw_varela_df = clean_df(build_df('1s-qn2puLikOtN1YGOX2WZfZrNMxA6K4e', 'raw_varela_data.csv'))

stylo_scores_df = build_df('1g4zAbrxtVSqJ1GwwXKJufqMHXpWSrLsC', 'stylo_scores.csv', index_col=0)
tfidf_scores_df = build_df('12RMTasYFx6WAi-OByHaTFI0_wRRTQytX', 'idf_scores.csv', index_col=0)

raw_news_df.pop('Link')
raw_varela_df.pop('Subject')


def remove_varela_authors(df):
  regex_baleia = 'baleia - \d{1,2}\/\d{1,2}\/\d{2,4}'
  regex_ana = 'ana cristina cavalcante\s*\d{1,2} \w{1,4} \d{2,4} - \S{3,5}(min)*'
  regex_adriano = 'adriano gambarini - \d{1,2}\/\d{1,2}\/\d{2,4}'
  regex_ivolnildo = 'ivon(i)*l(d)*o lavôr(\s)*\d{1,2} \w{1,4} \d{2,4} - \S{4,5}(min)*'
  regex_mario = 'mário pinto(\s)*\d{1,2} \w{1,4} \d{2,4} - \S{4,5}(min)*'
  regex_julio = "mais sobr julio preuss - \d{1,2}\/\d{1,2}\/\d{2,4}(julio preussescreveu o livro 'fotografia digital: da compra da câmera à impressão das fotos)*"
  regex_roberto = 'roberto linsker - \d{1,2}\/\d{1,2}\/\d{2,4}'
  df['Text'] = df['Text'].str.replace(regex_baleia, ' ')
  df['Text'] = df['Text'].str.replace(regex_ana, ' ')
  df['Text'] = df['Text'].str.replace(regex_adriano, ' ')
  df['Text'] = df['Text'].str.replace(regex_ivolnildo, ' ')
  df['Text'] = df['Text'].str.replace(regex_mario, ' ')
  df['Text'] = df['Text'].str.replace(regex_julio, ' ')
  df['Text'] = df['Text'].str.replace(regex_roberto, ' ')
  return df

raw_varela_df = remove_varela_authors(raw_varela_df)


In [4]:
# Remove few class entries (We need at least 3 samples of an author) and encode Author column 

def remove_entries(df, size):
  return df.groupby('Author').filter(lambda x: len(x) > size)

def encode_target_column(df, column):
  le = LabelEncoder()
  y = df.pop(column)
  encoded_Y = le.fit_transform(y)
  return encoded_Y

df = remove_entries(df, 2)
raw_df = remove_entries(raw_df, 2)

encoded_Y = encode_target_column(df, 'Author')
encoded_raw_Y  = encode_target_column(raw_df, 'Author')

encoded_Y_news  = encode_target_column(news_df, 'Author')
raw_news_df.pop('Author')

varela_Y_backup = varela_df['Author']
encoded_varela_Y = encode_target_column(varela_df, 'Author')
raw_varela_df.pop('Author')


corpus = raw_df['Text']
corpus_news = raw_news_df['Text']
corpus_varela = raw_varela_df['Text']

In [ ]:
df.columns

In [5]:
# Prepare our training and test sets
X_varela, X_varela_test, Y_varela, Y_varela_test = train_test_split(varela_df, encoded_varela_Y, random_state=7, test_size=0.3, stratify=encoded_varela_Y)
X_news, X_news_test, Y_news, Y_news_test = train_test_split(news_df, encoded_Y_news, random_state=7, test_size=0.3, stratify=encoded_Y_news)
X_student, X_student_test, Y_student, Y_student_test = train_test_split(df, encoded_Y, random_state=7, test_size=0.3, stratify=encoded_Y)

X_student_idf , X_student_idf_test, Y_student_idf, Y_student_idf_test = train_test_split(TfidfVectorizer().fit_transform(raw_df['Text']), encoded_Y, random_state=7, test_size=0.25, stratify=encoded_Y)
X_news_idf , X_news_idf_test, Y_news_idf, Y_news_idf_test = train_test_split(TfidfVectorizer().fit_transform(raw_news_df['Text']), encoded_Y_news, random_state=7, test_size=0.3, stratify=encoded_Y_news)
X_varela_idf , X_varela_idf_test, Y_varela_idf, Y_varela_idf_test = train_test_split(TfidfVectorizer().fit_transform(raw_varela_df['Text']), encoded_varela_Y, random_state=7, test_size=0.3, stratify=encoded_varela_Y)

df['Author'] = encoded_Y
news_df['Author'] = encoded_Y_news
varela_df['Author'] = encoded_varela_Y

In [6]:
def train_and_report(classifier, X, X_test, Y, Y_test, dataset_name, average='macro', verbose=True):

  clf = classifier.fit(X, Y)
  y_pred = clf.predict(X_test)
  print(dataset_name)
  acc = accuracy_score(Y_test, y_pred)
  print('ACCURACY:', acc)
  roc_auc_dict = roc_auc_score_multiclass(Y_test, y_pred, average=average)
  print('ROC AUC MEAN:', np.mean(list(roc_auc_dict.values())))

  if verbose:
    print('BLA')

  return np.mean(list(roc_auc_dict.values())), acc

# Verification evaluation functions

def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict


def classification_report(y_true, y_pred, y_score=None, average='micro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred'] = pred_cnt
    class_report_df['pred'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df

In [9]:

model_names = ['RL', 'NB Gaus', 'Random Forest','Árvores Extra','MLP']
scaling_names = ['ns', 'ss', 'mm', 'pt']

def build_metrics_df(model_names, metrics, scales):
  rows = pd.Series(model_names)
  cols = pd.Series(metrics)
  idx = pd.MultiIndex.from_product([rows, scales])
  return pd.DataFrame(index=idx, columns=cols)

test = build_metrics_df(model_names, metrics, scaling_names)

In [10]:
models = [LogisticRegression(random_state=0), GaussianNB(), 
          RandomForestClassifier(random_state=0), ExtraTreesClassifier(random_state=0), 
          MLPClassifier(random_state=0)]

scalers = [StandardScaler(), StandardScaler(), MinMaxScaler(), PowerTransformer()]
metrics = ['acurácia', 'roc auc']

def set_scores(df, model_name, scale, acc, auc):
  df.loc[model_name, scale]['acurácia'] = acc
  df.loc[model_name, scale]['roc auc'] = auc

stylo_data = [
   [X_student, X_student_test, Y_student, Y_student_test],
   [X_news, X_news_test, Y_news, Y_news_test], 
   [X_varela, X_varela_test, Y_varela, Y_varela_test] ]

tfidf_data = [
   [X_student_idf.toarray(), X_student_idf_test.toarray(), Y_student_idf, Y_student_idf_test],
   [X_news_idf.toarray(), X_news_idf_test.toarray(), Y_news_idf, Y_news_idf_test], 
   [X_varela_idf.toarray(), X_varela_idf_test.toarray(), Y_varela_idf, Y_varela_idf_test] ]

def scaling(scaler, data):
  return [scaler.fit_transform(X) for X in data]

#TODO: Once that is working, figure out the best way of running gridSearchCV on each model for each scaling technique
def run_classic_models(data):
  student_stylo_acc_df = build_metrics_df(model_names, metrics, scaling_names)
  news_stylo_acc_df = student_stylo_acc_df.copy()
  varela_stylo_acc_df = student_stylo_acc_df.copy()
  for idx, scaler in enumerate(scalers):
    print(scaling_names[idx])
    if scaling_names[idx] != 'ns':
      scaled_data = scaling(scaler, [data[0][0], data[0][1], data[1][0], data[1][1], data[2][0], data[2][1]])
    else:
      print('no scaling')
      scaled_data = [data[0][0], data[0][1], data[1][0], data[1][1], data[2][0], data[2][1]]

    for model, model_name in zip(models, model_names):
      auc, acc = train_and_report(model, scaled_data[0], scaled_data[1], data[0][2], data[0][3], 'Base de estudantes', average="macro", verbose=False)
      set_scores(student_stylo_acc_df, model_name, scaling_names[idx], acc, auc)
      
      auc, acc = train_and_report(model, scaled_data[2], scaled_data[3], data[1][2], data[1][3], 'Base de notícias', average="macro", verbose=False)
      set_scores(news_stylo_acc_df, model_name, scaling_names[idx], acc, auc)
        
      auc, acc = train_and_report(model, scaled_data[4], scaled_data[5], data[2][2], data[2][3], 'Base Varela', average="macro", verbose=False )
      set_scores(varela_stylo_acc_df, model_name, scaling_names[idx], acc, auc)

  return student_stylo_acc_df, news_stylo_acc_df, varela_stylo_acc_df  


In [ ]:
build_metrics_df(model_names, metrics, scaling_names)

In [11]:
deep_model_names = ['NN']
metrics = ['acurácia', 'roc auc macro', 'roc auc mic']  

def set_deep_scores(df, model_name, scale, acc, roc_auc_mac, roc_auc_mic):
  df.loc[model_name, scale]['acurácia'] = acc
  df.loc[model_name, scale]['roc auc macro'] = roc_auc_mac
  df.loc[model_name, scale]['roc auc mic'] = roc_auc_mic

def build_large_nn(input_size, num_of_classes):
    print('input size:{}, num of classes:{}'.format(input_size, num_of_classes))
    model = Sequential()
    model.add(Dense(160, input_dim=input_size, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(80, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(40, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def build_small_nn(input_size, num_of_classes):
    print('input size:{}, num of classes:{}'.format(input_size, num_of_classes))
    model = Sequential()
    model.add(Dense(64, input_dim=input_size, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation='selu'))
    model.add(Dense(num_of_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def binarize_split_scale(X, Y, scaler, scaler_name):
    lb = LabelBinarizer() 
    y = lb.fit_transform(Y)
    print(y.shape)#(84, 16)
    n_classes = y.shape[1]      
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=0, test_size=0.3, stratify=y)

    if scaler_name != 'ns':
      scaled_data = scaling(scaler, [X_train, X_test])
    else:
      print('no scaling')
      scaled_data = [X_train, X_test]
    
    return scaled_data, Y_train, Y_test, n_classes
    

def roc_auc_multiclass_nn(X_train, X_test, Y_train, Y_test, n_classes, epochs, columns=True):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    acc = 0
    roc_auc = dict()
  
    nn_input_size = X_train.shape[1]
    if columns:
      estimator = build_small_nn(nn_input_size, n_classes)
    else:
      estimator = build_large_nn(nn_input_size, n_classes)
      
    #estimator = KerasClassifier(build_fn=build_nn, input_size=nn_input_size, num_of_classes=len(set(Y)),  epochs=200, batch_size=128)
    hist = estimator.fit(X_train, Y_train, verbose=0, epochs=epochs, validation_data=(X_test, Y_test))

    print('finished training')
    y_pred_keras = estimator.predict(X_test)

    for i in range(n_classes):
      fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], y_pred_keras[:, i])
      
      if (np.isnan(sum(tpr[i]))):
        tpr[i] = [0.5] * len(tpr[i])

      if (np.isnan(sum(fpr[i]))):
        fpr[i] = [0.5] * len(fpr[i])
      
      roc_auc[i] = auc(fpr[i], tpr[i])
      # Compute micro-average ROC curve and ROC area
      fpr["micro"], tpr["micro"], _ = roc_curve(Y_test.ravel(), y_pred_keras.ravel())
      roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Compute macro-average ROC curve and ROC area
    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    print(roc_auc)
    return roc_auc, tpr, fpr, hist

def run_deep_models(data, cols=True):
  student_stylo_acc_df = build_metrics_df(deep_model_names, metrics, scaling_names)
  news_stylo_acc_df = student_stylo_acc_df.copy()
  varela_stylo_acc_df = student_stylo_acc_df.copy()

  for idx, scaler in enumerate(scalers):
    print(scaling_names[idx])

    scaled_data, Y, Y_test, n_classes = binarize_split_scale(data[0][0], data[0][1], scaler , scaling_names[idx])
    roc, _, _, hist_s = roc_auc_multiclass_nn(scaled_data[0], scaled_data[1], Y, Y_test, n_classes, 200, cols)
    set_deep_scores(student_stylo_acc_df, 'NN', scaling_names[idx], hist_s.history['val_accuracy'][-1], roc["macro"], roc["micro"])

    scaled_data, Y, Y_test, n_classes = binarize_split_scale(data[1][0], data[1][1], scaler, scaling_names[idx])
    roc, _, _, hist_n = roc_auc_multiclass_nn(scaled_data[0], scaled_data[1], Y, Y_test, n_classes, 200, cols)
    set_deep_scores(news_stylo_acc_df, 'NN', scaling_names[idx], hist_n.history['val_accuracy'][-1], roc["macro"], roc["micro"])

    scaled_data, Y, Y_test, n_classes = binarize_split_scale(data[2][0], data[2][1], scaler , scaling_names[idx])
    roc, _, _, hist_v = roc_auc_multiclass_nn(scaled_data[0], scaled_data[1], Y, Y_test, n_classes, 200, cols)
    set_deep_scores(varela_stylo_acc_df, 'NN', scaling_names[idx], hist_v.history['val_accuracy'][-1], roc["macro"], roc["micro"])


  return student_stylo_acc_df, news_stylo_acc_df, varela_stylo_acc_df, hist_s, hist_n, hist_v

nn_stylo_data = [
           [df, encoded_Y],
           [news_df, encoded_Y_news],
           [varela_df, encoded_varela_Y]
]

nn_tfidf_data = [
   [TfidfVectorizer(binary=True).fit_transform(raw_df['Text']).toarray(), encoded_Y],
   [TfidfVectorizer(binary=True).fit_transform(raw_news_df['Text']).toarray(), encoded_Y_news], 
   [TfidfVectorizer(binary=True).fit_transform(raw_varela_df['Text']).toarray(), encoded_varela_Y]
]

#std, news, var, hs, hn, hv = run_deep_models(nn_stylo_data, True)
#std_idf, news_idf, var_idf, hs_idf, hn_idf, hv_idf = run_deep_models(nn_tfidf_data, False)

In [ ]:
std.merge(news, left_index=True, right_index=True, suffixes=('_estudantes', '_notícias')).merge(var, left_index=True, right_index=True).to_csv('nn-stylo-scaled.csv')

In [ ]:
std_idf.merge(news_idf, left_index=True, right_index=True, suffixes=('_estudantes', '_notícias')).merge(var_idf, left_index=True, right_index=True).to_csv('nn-idf-scaled.csv')

In [ ]:
student_df, news_df, varela_df = run_classic_models(tfidf_data)

In [ ]:
student_df.merge(news_df, left_index=True, right_index=True, suffixes=('_estudantes', '_notícias')).merge(varela_df, left_index=True, right_index=True).to_csv('scaled-stylo-score.csv')

In [ ]:
student_idf_df, news_idf_df, varela_idf_df = run_classic_models(tfidf_data)

In [ ]:
student_idf_df.merge(news_idf_df, left_index=True, right_index=True, suffixes=('_estudantes', '_notícias')).merge(varela_idf_df, left_index=True, right_index=True)

In [ ]:
student_idf_df.merge(news_idf_df, left_index=True, right_index=True, suffixes=('_estudantes', '_notícias')).merge(varela_idf_df, left_index=True, right_index=True).to_csv('scaled-idf-score.csv')

In [ ]:
scaled_df.to_latex()

In [ ]:
def merge_and_build_acc_df(students_df, news_df, varela_df):
    merged_df = students_df.merge(news_df, left_index=True, right_index=True, suffixes=('_estudantes', '_notícias')).merge(varela_df, left_index=True, right_index=True)
    return build_acc_df(merged_df)

def merge_and_build_auc_df(students_df, news_df, varela_df):
    merged_df = students_df.merge(news_df, left_index=True, right_index=True, suffixes=('_estudantes', '_notícias')).merge(varela_df, left_index=True, right_index=True)
    return build_auc_df(merged_df)

def build_acc_df(df, nn=False):
    merged_df = df.copy()
    merged_df.drop(columns='roc auc_estudantes', inplace=True)
    merged_df.drop(columns='roc auc_notícias', inplace=True)
    merged_df.drop(columns='roc auc', inplace=True)
    merged_df.rename(columns={'acurácia_estudantes': 'Estudantes', 'acurácia_notícias' : 'Notícias', 'acurácia':'Varela'}, inplace=True)
    return merged_df

def build_auc_df(df, nn=False):
    merged_df = df.copy()
    merged_df.drop(columns='acurácia_estudantes', inplace=True)
    merged_df.drop(columns='acurácia_notícias', inplace=True)
    merged_df.drop(columns='acurácia', inplace=True)
    merged_df.rename(columns={'roc auc_estudantes': 'Estudantes', 'roc auc_notícias' : 'Notícias', 'roc auc':'Varela'}, inplace=True)
    return merged_df


In [ ]:
merged_df = student_acc.merge(news_acc, left_index=True, right_index=True, suffixes=('_estudantes', '_notícias')).merge(varela_acc, left_index=True, right_index=True)
merged_df.drop(columns='roc auc_estudantes', inplace=True)
merged_df.drop(columns='roc auc_notícias', inplace=True)
merged_df.drop(columns='roc auc', inplace=True)
merged_df.rename(columns={'acurácia_estudantes': 'Estudantes', 'acurácia_notícias' : 'Notícias', 'acurácia':'Varela'}, inplace=True)

merged_df

In [ ]:

stylo_scores_df = build_df('1_mnFalKzNJcZNnBnttON3HHoNTv6s4lS', 'stylo_scores.csv', index_col=0)
tfidf_scores_df = build_df('12RMTasYFx6WAi-OByHaTFI0_wRRTQytX', 'idf_scores.csv', index_col=0)

stylo_scores_scaled_df = build_df('1W-WCQ1ATEQErG2Mv3Vs3wEVCXJtdo42m', 'stylo_scores.csv', index_col=[0,1])
tfidf_scores_scaled_df = build_df('1hH5i3Gk5_Z_FztWmSk4G4M20_nY37gz8', 'idf_scores.csv', index_col=[0,1])

nn_stylo_scaled_df = build_df('1z5nQ5HrvB27AnV8mH7tBDf4RaeBek3n3', 'nn_stylo_scores.csv', index_col=[0,1])
nn_idf_scaled_df = build_df('1S2S31meoaQeJbWlU2OHmLhlLGZZaqaKL', 'nn_stylo_scores.csv', index_col=[0,1])

In [ ]:
nn_stylo_scaled_df.rename(columns={'roc auc mic_estudantes': 'roc auc_estudantes', 
                                   'roc auc mic_notícias': 'roc auc_notícias', 
                                   'roc auc mic': 'roc auc'}, inplace=True)

nn_idf_scaled_df.rename(columns={'roc auc mic_estudantes': 'roc auc_estudantes', 
                                   'roc auc mic_notícias': 'roc auc notícias', 
                                   'roc auc mic': 'roc auc'}, inplace=True)

In [ ]:
stylo_scores_df.rename(columns={'acurácia': 'acurácia_varela', 'roc auc': 'roc auc_varela'}, inplace=True)
tfidf_scores_df.rename(columns={'acurácia': 'acurácia_varela', 'roc auc': 'roc auc_varela'}, inplace=True)

In [ ]:
stylo_scores_scaled_df

In [ ]:
stylo_scores_df

,acurácia_estudantes,roc auc_estudantes,acurácia_notícias,roc auc_notícias,acurácia_varela,roc auc_varela
RL,0.153846,0.544461,0.500000,0.722222,0.187778,0.589787
NB Gaus,0.076923,0.494552,0.833333,0.907407,0.555556,0.775533
RF,0.115385,0.511313,0.766667,0.870370,0.561111,0.778339
Árvores Extra,0.153846,0.543918,0.833333,0.907407,0.572222,0.783951
MLP,0.115385,0.532751,0.400000,0.666667,0.244444,0.618406
NN,0.095238,0.584127,0.120000,0.584889,0.009333,0.860000


In [ ]:
stylo_scores_scaled_df.loc[('RL','Sem normalização'),:] = stylo_scores_df.loc['RL']
tfidf_scores_scaled_df.loc[('RL','Sem normalização'),:] = tfidf_scores_df.loc['RL']

stylo_scores_scaled_df.loc[('NB Gaus','Sem normalização'),:] = stylo_scores_df.loc['NB Gaus']
tfidf_scores_scaled_df.loc[('NB Gaus','Sem normalização'),:] = tfidf_scores_df.loc['NB Gaus']

stylo_scores_scaled_df.loc[('Random Forest','Sem normalização'),:] = stylo_scores_df.loc['RF']
tfidf_scores_scaled_df.loc[('Random Forest','Sem normalização'),:] = tfidf_scores_df.loc['RF']

stylo_scores_scaled_df.loc[('Árvores Extra','Sem normalização'),:] = stylo_scores_df.loc['Árvores Extra']
tfidf_scores_scaled_df.loc[('Árvores Extra','Sem normalização'),:] = tfidf_scores_df.loc['Árvores Extra']

stylo_scores_scaled_df.loc[('MLP','Sem normalização'),:] = stylo_scores_df.loc['MLP']
tfidf_scores_scaled_df.loc[('MLP','Sem normalização'),:] = tfidf_scores_df.loc['MLP']

In [ ]:
stylo_scores_scaled_df

acurácia_estudantes  ...  roc auc_varela
RL            Sem normalização              0.095238  ...        0.860000
              Standard Scaler               0.153846  ...        0.801347
              Min Max                       0.230769  ...        0.756453
              Power Transformer             0.230769  ...        0.804714
NB Gaus       Sem normalização              0.115385  ...        0.775533
              Standard Scaler               0.115385  ...        0.514590
              Min Max                       0.076923  ...        0.562851
              Power Transformer             0.038462  ...        0.514590
Random Forest Sem normalização              0.115385  ...        0.778339
              Standard Scaler               0.115385  ...        0.778339
              Min Max                       0.153846  ...        0.722222
              Power Transformer             0.115385  ...        0.782828
Árvores Extra Sem normalização              0.076923  ...        0.783951
              Standard Scaler               0.115385  ...        0.782828
              Min Max                       0.115385  ...        0.713244
              Power Transformer             0.192308  ...        0.795174
MLP           Sem normalização              0.192308  ...        0.618406
              Standard Scaler               0.192308  ...        0.796296
              Min Max                       0.115385  ...        0.776094
              Power Transformer             0.192308  ...        0.809764
NN            Sem normalização              0.095238  ...        0.860000
              Standard Scaler               0.192308  ...        0.970950
              Min Max                       0.153846  ...        0.944678
              Power Transformer             0.076923  ...        0.975654

[24 rows x 6 columns]

In [ ]:
#new_index = pd.MultiIndex.from_product([['ML'], ['ns','ss', 'mm', 'pt']])

In [ ]:
# Adding new indexes to our multiindex
stylo_scores_scaled_df.loc[('NN','ns'),:] = stylo_scores_df.loc['NN']
stylo_scores_scaled_df.loc[('NN','ss'),:] = 0
stylo_scores_scaled_df.loc[('NN','mm'),:] = 0
stylo_scores_scaled_df.loc[('NN','pt'),:] = 0

tfidf_scores_scaled_df.loc[('NN','ns'),:] = tfidf_scores_df.loc['NN']
tfidf_scores_scaled_df.loc[('NN','ss'),:] = 0
tfidf_scores_scaled_df.loc[('NN','mm'),:] = 0
tfidf_scores_scaled_df.loc[('NN','pt'),:] = 0

In [ ]:

tfidf_scores_scaled_df.to_csv('tf-idf-scaled-scores.csv')
stylo_scores_scaled_df.to_csv('stylo-scaled-scores.csv')

In [ ]:
stylo_scores_scaled_df.rename(index={"Sem scaling": "Sem normalização", "ss": "Standard Scaler", "mm": "Min Max", "pt": "Power Transformer"}, inplace=True)
tfidf_scores_scaled_df.rename(index={"Sem scaling": "Sem normalização", "ss": "Standard Scaler", "mm": "Min Max", "pt": "Power Transformer"}, inplace=True)

In [ ]:
stylo_scores_scaled_df.loc['NN', 'ss']  = nn_stylo_scaled_df.loc['NN', 'ss']
stylo_scores_scaled_df.loc['NN', 'mm']  = nn_stylo_scaled_df.loc['NN', 'mm']
stylo_scores_scaled_df.loc['NN', 'pt']  = nn_stylo_scaled_df.loc['NN', 'pt']

tfidf_scores_scaled_df.loc['NN', 'ss']  = nn_idf_scaled_df.loc['NN', 'ss']
tfidf_scores_scaled_df.loc['NN', 'mm']  = nn_idf_scaled_df.loc['NN', 'mm']
tfidf_scores_scaled_df.loc['NN', 'pt']  = nn_idf_scaled_df.loc['NN', 'pt']

In [ ]:
tfidf_scores_scaled_df.rename(columns={'acurácia': 'acurácia_varela', 'roc auc': 'roc auc_varela'}, inplace=True)
stylo_scores_scaled_df.rename(columns={'acurácia': 'acurácia_varela', 'roc auc': 'roc auc_varela'}, inplace=True)

In [ ]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [ ]:
base_value_student = tfidf_scores_scaled_df.loc['RL'].loc['Sem normalização']['acurácia_estudantes']
base_value_news = tfidf_scores_scaled_df.loc['RL'].loc['Sem normalização']['acurácia_notícias']
base_value_var = tfidf_scores_scaled_df.loc['RL'].loc['Sem normalização']['acurácia_varela']

In [ ]:
stylo_html = stylo_scores_scaled_df.style.highlight_max(color = 'lightgreen', axis = 0)

In [ ]:
tfidf_html = tfidf_scores_scaled_df.style.highlight_max(color = 'lightgreen', axis = 0)


In [ ]:
stylo_html.render()

'<style  type="text/css" >\n#T_10031b48_51e1_11eb_a42b_0242ac1c0002row2_col0,#T_10031b48_51e1_11eb_a42b_0242ac1c0002row3_col0,#T_10031b48_51e1_11eb_a42b_0242ac1c0002row19_col4,#T_10031b48_51e1_11eb_a42b_0242ac1c0002row22_col1,#T_10031b48_51e1_11eb_a42b_0242ac1c0002row23_col2,#T_10031b48_51e1_11eb_a42b_0242ac1c0002row23_col3,#T_10031b48_51e1_11eb_a42b_0242ac1c0002row23_col5{\n            background-color:  lightgreen;\n        }</style><table id="T_10031b48_51e1_11eb_a42b_0242ac1c0002" ><thead>    <tr>        <th class="blank" ></th>        <th class="blank level0" ></th>        <th class="col_heading level0 col0" >acurácia_estudantes</th>        <th class="col_heading level0 col1" >roc auc_estudantes</th>        <th class="col_heading level0 col2" >acurácia_notícias</th>        <th class="col_heading level0 col3" >roc auc_notícias</th>        <th class="col_heading level0 col4" >acurácia_varela</th>        <th class="col_heading level0 col5" >roc auc_varela</th>    </tr></thead><tbody>

In [ ]:
def highlight_min(data):
    color= 'lightgreen'
    attr = 'background-color: {}'.format(color)
    print(data.ndim)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else: 
        is_max = data.groupby(level=0).transform('max') == data
        return pd.DataFrame(np.where(is_max, attr, ''), index=data.index, columns=data.columns)
        

highlight_df = tfidf_scores_scaled_df.style.apply(highlight_min, axis=None)


In [ ]:
stylo_scores_scaled_df.loc['NN'].loc['Sem normalização']['roc auc_varela'] = 0.5

In [ ]:
highlight_df_stylo = stylo_scores_scaled_df.style.apply(highlight_min, axis=None)

In [ ]:
style = tfidf_scores_scaled_df.style.highlight_min(axis=0)

In [ ]:
highlight_df.render()

2


'<style  type="text/css" >\n#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row0_col2,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row0_col3,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row2_col2,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row2_col3,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row2_col4,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row2_col5,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row3_col0,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row3_col1,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row6_col0,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row6_col1,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row6_col2,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row6_col3,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row7_col4,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row7_col5,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row8_col2,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row8_col3,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row8_col4,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row8_col5,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002row9_col0,#T_f13a999c_51c8_11eb_a42b_0242ac1c0002ro

In [ ]:
highlight_min(tfidf_scores_scaled_df)

In [ ]:
rl_df.style.highlight_max(color = 'lightgreen', axis = 0)


,acurácia_estudantes,roc auc_estudantes,acurácia_notícias,roc auc_notícias,acurácia_varela,roc auc_varela
Sem normalização,0.095238,0.500000,0.833333,0.907407,0.762222,0.879910
Standard Scaler,0.095238,0.500000,0.800000,0.888889,0.788889,0.893378
Min Max,0.142857,0.517352,0.833333,0.907407,0.864444,0.931538
Power Transformer,0.190476,0.534539,0.766667,0.870370,0.776667,0.887205


In [ ]:
test_df = stylo_scores_scaled_df.copy()

#test_df.loc['RL']['ns'] = 0
test = test_df.index.insert(0, ('RL','ns'))
test = test.insert(4, ('NB Gaus','ns'))
test = test.insert(8, ('Random Forest','ns'))
test = test.insert(12, ('Árvores Extra','ns'))
test = test.insert(16, ('MLP','ns'))

# # Convert index to dataframe
# old_idx = test_df.index.to_frame()

# # Insert new level at specified location
# old_idx.insert(0, 'new_level_name', new_level_values)

# # Convert back to MultiIndex
# df.index = pandas.MultiIndex.from_frame(old_idx)
#df.set_index('Firstlevel', append=True, inplace=True)

In [12]:
def build_tree_params():
  n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
  # Number of features to consider at every split
  max_features = ['auto', 'sqrt']
  # Maximum number of levels in tree
  max_depth = [int(x) for x in np.linspace(5, 110, num = 10)]
  max_depth.append(None)
  # Minimum number of samples required to split a node
  min_samples_split = [2, 5, 10]
  # Minimum number of samples required at each leaf node
  min_samples_leaf = [1, 2, 4]
  # Method of selecting samples for training each tree
  bootstrap = [True, False]
  # Create the random grid
  random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

  return random_grid

def build_mlp_params():
   activation = ['identity', 'logistic', 'tanh', 'relu']
   alpha = [0.0001, 0.001, 0.01, 0.1]
   hidden_layer_sizes = [(50,), (100,), (150,),(200,)]
   learning_rate = ['constant', 'invscaling', 'adaptive']
   max_iterint = [100, 200, 300, 400, 500]
   random_grid = {'activation': activation,
               'alpha': alpha,
               'hidden_layer_sizes': hidden_layer_sizes,
               'learning_rate': learning_rate,
               'max_iterint': max_iterint}
   return random_grid

def build_lr_params():
  penalty=['l1','l2']
  c = [0.1, 0.5, 1, 2, 5]
  solver=['newton-cg', 'lbfgs','lib','saga']
  max_iterint = [25, 50, 100, 200, 500]
  random_grid = {'penalty': penalty,
               'c': c,
               'solver': solver,
               'max_iterint': max_iterint
               }
  return random_grid

def build_nb_params():
  var_smoothing = [1e-05, 1e-08, 1e-09, 1e-10]
  random_grid = {'var_smoothing': var_smoothing}
  return random_grid

def build_nn_params():
  optimizer=['adam', 'sgd', 'rmsprop','adamax']
  output_activation_function = ['relu', 'selu', 'softmax', 'tanh']
  epochs = [100, 200, 300, 400, 500]
  kernel_initializer = ['randomuniform', 'glorotnormal', 'orthogonal']
  random_grid = {'optimizer': optimizer,
               'output_activation_function': output_activation_function,
               'epochs': epochs,
               'kernel_initializer': kernel_initializer,
               }
  return random_grid

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV


def optimize(X, Y, models, params, scaler=None):
  best_models = []
  for idx, model in enumerate(models):

    best_student = cv_optimize(X[0], Y[0], model, params[idx], 'student')
    best_news = cv_optimize(X[1], Y[1], model, params[idx], 'news')
    best_varela = cv_optimize(X[2], Y[2], model, params[idx], 'varela')
    best_models.append([best_student, best_news, best_varela])
  return best_models


def cv_optimize(X, Y, model, optimization_params, dataset_title, scaler=None):
  if scaler is not None:
    X = scaler.fit_transform(X)

  cv_results = cross_validate(model, X, Y, cv=3)
  print('base {} cv score:{}'.format(dataset_title, cv_results['test_score']))

  rf_random = RandomizedSearchCV(estimator = model, param_distributions = optimization_params, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
  rf_random.fit(X, Y)
  print(rf_random.best_params_)

  optimal_cv_results = cross_validate(rf_random.best_estimator_, X, Y, cv=3)
  print('optimal {} cv score:{}'.format(dataset_title, optimal_cv_results['test_score']))
  
  print('overall gain after optimization:{}'( np.mean(optimal_cv_results) - np.mean(cv_results)))
  return rf_random.best_estimator_


best_models_re_exp1 = optimize([df,news_df, varela_df], [encoded_Y, encoded_Y_news, encoded_varela_Y],                      
              [GaussianNB(), RandomForestClassifier(random_state=0)],
              [build_nb_params(), build_tree_params()],
              scaler=None
            )

best_models_re_exp2 = optimize([df,news_df, varela_df], [encoded_Y, encoded_Y_news, encoded_varela_Y], 
             [MLPClassifier(random_state=0)],
             [build_mlp_params()], scaler=StandardScaler())


#TODO: Test and investigate how it will work for the Keras NN (maybe wrapping around a KerasClassifier..)
best_models_re_exp3 = optimize([df,news_df, varela_df], [encoded_Y, encoded_Y_news, encoded_varela_Y], 
             [ExtraTrees(random_state=0), LogisticRegression(random_state=0), build_nn()],
             [build_tree_params(), build_lr_params(), build_nn_params()], 
             scaler=PowerTransformer())


best_models_rt_exp1 = optmize([TfidfVectorizer().fit_transform(raw_df['Text']),
              TfidfVectorizer().fit_transform(raw_news_df['Text']), 
              TfidfVectorizer().fit_transform(raw_varela_df['Text'])],
              [encoded_Y, encoded_Y_news, encoded_varela_Y],
              [RandomForestClassifier(random_state=0), MLPClassifier(random_state=0)],
              [build_tree_params(), build_mlp_params()],
              scaler=None
            )

best_models_rt_exp2 = optimize([TfidfVectorizer().fit_transform(raw_df['Text']),
              TfidfVectorizer().fit_transform(raw_news_df['Text']), 
              TfidfVectorizer().fit_transform(raw_varela_df['Text'])],
              [encoded_Y, encoded_Y_news, encoded_varela_Y],
              [ExtraTrees(random_state=0)],
              [build_tree_params()],
              scaler=scaler=StandardScaler())


#TODO: Test and investigate how it will work for the Keras NN (maybe wrapping around a KerasClassifier..)
best_models_rt_exp3 = optmize([TfidfVectorizer().fit_transform(raw_df['Text']),
              TfidfVectorizer().fit_transform(raw_news_df['Text']), 
              TfidfVectorizer().fit_transform(raw_varela_df['Text'])],
              [encoded_Y, encoded_Y_news, encoded_varela_Y],
              [ExtraTrees(random_state=0), LogisticRegression(random_state=0), build_nn()],
              [build_tree_params(), build_lr_params(), build_nn_params()], 
              scaler=scaler=MinMaxScaler())




base student cv score:[0.14285714 0.         0.17857143]
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   56.8s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.4min finished


{'n_estimators': 600, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 98, 'bootstrap': True}
optimal student cv score:[0.21428571 0.         0.32142857]
base news cv score:[0.79411765 0.84848485 0.78787879]
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   55.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.4min finished


{'n_estimators': 800, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 75, 'bootstrap': False}
optimal news cv score:[0.79411765 0.75757576 0.87878788]
base varela cv score:[0.535 0.554 0.532]
Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 20.4min


In [18]:
df.drop(columns=['Author'], inplace=True)
news_df.drop(columns=['Author'], inplace=True)
varela_df.drop(columns=['Author'], inplace=True)



In [20]:
encoded_varela_Y

array([38, 38, 38, ..., 82, 82, 82])

In [14]:
X_varela, X_varela_test, Y_varela, Y_varela_test = train_test_split(varela_df, encoded_varela_Y, random_state=7, test_size=0.3, stratify=encoded_varela_Y)
X_news, X_news_test, Y_news, Y_news_test = train_test_split(news_df, encoded_Y_news, random_state=7, test_size=0.3, stratify=encoded_Y_news)
X_student, X_student_test, Y_student, Y_student_test = train_test_split(df, encoded_Y, random_state=7, test_size=0.3, stratify=encoded_Y)

X_student_idf , X_student_idf_test, Y_student_idf, Y_student_idf_test = train_test_split(TfidfVectorizer().fit_transform(raw_df['Text']), encoded_Y, random_state=7, test_size=0.25, stratify=encoded_Y)
X_news_idf , X_news_idf_test, Y_news_idf, Y_news_idf_test = train_test_split(TfidfVectorizer().fit_transform(raw_news_df['Text']), encoded_Y_news, random_state=7, test_size=0.3, stratify=encoded_Y_news)
X_varela_idf , X_varela_idf_test, Y_varela_idf, Y_varela_idf_test = train_test_split(TfidfVectorizer().fit_transform(raw_varela_df['Text']), encoded_varela_Y, random_state=7, test_size=0.3, stratify=encoded_varela_Y)


(58, 74)